<a href="https://colab.research.google.com/github/guico3lho/NLP_Projeto_Final/blob/main/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras
!pip install tensorflow
!pip install opendatasets


guico3lhoLooking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import re
import opendatasets as od
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# objetivo é eu prever o valor de uma ação com base nas noticias
# cada endereço vai receber 25 noticias concatenadas e cada encereço corresponde a uma data
# corresponde a um registro


In [4]:
#%% Functions
def normalizarStringStockHeadlines(text):
    import unicodedata

    text = text.lower()
    text = re.sub(r'[\\\']', '', text)
    # text = re.sub("r'\'\''), '', text)"
    # remove b' from the text

    return text

In [5]:
#%% Importing the data
od.download("https://www.kaggle.com/datasets/lykin22/stock-headlines")



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: guico3lho
Your Kaggle Key: ··········


100%|██████████| 3.18M/3.18M [00:00<00:00, 188MB/s]

In [6]:
df_raw = pd.read_csv('/content/stock-headlines/Stock Headlines.csv', encoding='ISO-8859-1')
df = df_raw.copy()

In [7]:
#%% Dropping NaN values

df.dropna(inplace=True)

df.reset_index(inplace=True)

In [8]:
#%% Concatenating all the news of a specific day to a single column

headlines = []
for i in range(0,len(df)):
        headlines.append(' '.join(str(x) for x in df.iloc[i,3:]))
        ...

df['Headlines'] = headlines

In [9]:
#%% Normalizing the text
df['Headlines'] = df['Headlines'].apply(lambda x: normalizarStringStockHeadlines(x))

In [10]:
#%% Selecting columns for the model
df = df[['Label','Headlines']]

In [11]:
#%% Splitting the data
df_train,df_test = train_test_split(df, test_size=0.3, random_state=42)

df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=42)
...


Ellipsis

In [12]:
# categorizing the labels
df_train['Label'] = pd.Categorical(df_train['Label'])

y_train_int = df_train['Label'].cat.codes

df_valid['Label'] = pd.Categorical(df_valid['Label'])

y_valid_int = df_valid['Label'].cat.codes

df_test['Label'] = pd.Categorical(df_test['Label'])
y_test_int = df_test['Label'].cat.codes

y_train = to_categorical(y_train_int)
y_valid = to_categorical(y_valid_int)
y_test = to_categorical(y_test_int)




In [13]:
# y_train = df_train['Label'].to_numpy()
# y_valid = df_valid['Label'].to_numpy()
# y_test = df_test['Label'].to_numpy()


In [14]:

#%% Creating vocabular
vocabulary = {}
for i in range(0,len(df_train)):
    for word in df_train.iloc[i,1].split():
        if word not in vocabulary:
            vocabulary[word] = 1
        else:
            vocabulary[word] += 1

vocab_size = len(vocabulary)

In [15]:
df_train

,Label,Headlines
3920,1,japan furious at unesco listing nanjing massac...
4095,1,explosion at airport in istanbul yemeni former...
664,0,review: the wreck of the abergavenny by alethe...
3383,0,david cameron loses syria vote in commons docu...
2809,1,as the japanese government and tepco struggle ...
...,...,...
333,1,ipswich 2 - 1 man city comment: whitehall voic...
1547,1,eu protests at start of ethiopian treason tria...
3876,1,gps trackers in fake elephant tusks reveal ivo...
2754,0,anonymous open letter to the world [time to ma...


In [16]:
#%% Tokenizing the text

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

# why num_words? with or without it does not matter

# why pad_sequences not imported?
tokenizer = Tokenizer(num_words=len(vocabulary))
tokenizer.fit_on_texts(df_train['Headlines'])
word2index = tokenizer.word_index


In [17]:
# convert each sentence to a list of their respective word indexes
train_sequences = tokenizer.texts_to_sequences(df_train['Headlines'])
valid_sequences = tokenizer.texts_to_sequences(df_valid['Headlines'])
test_sequences = tokenizer.texts_to_sequences(df_test['Headlines'])

max_len = len(train_sequences[0])
for train,valid,test in zip(train_sequences,valid_sequences,test_sequences):
    if max(len(train),len(valid),len(test)) > max_len:
        max_len = max(len(train),len(valid),len(test))

print("maxlen:",max_len)

trunc_type = 'post'
padding_type = 'post'
max_len = 731

train_padded = pad_sequences(train_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(valid_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)


maxlen: 655


In [18]:
df

,Label,Headlines
0,0,a hindrance to operations: extracts from the l...
1,0,scorecard the best lake scene leader: german s...
2,0,coventry caught on counter by flo uniteds riva...
3,1,pilgrim knows how to progress thatcher facing ...
4,1,hitches and horlocks beckham off but united su...
...,...,...
4093,0,barclays and rbs shares suspended from trading...
4094,1,"2,500 scientists to australia: if you want to ..."
4095,1,explosion at airport in istanbul yemeni former...
4096,1,jamaica proposes marijuana dispensers for tour...


In [19]:
#%% Creating the model
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout,LSTM,Bidirectional
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Embedding(vocab_size, output_dim=max_len, input_length=max_len))
model.add(LSTM(256,return_sequences=True))
model.add(LSTM(128))
# model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(2, activation='softmax'))



model.summary()


...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 731, 731)          48531821  
                                                                 
 lstm (LSTM)                 (None, 731, 256)          1011712   
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 2)                 6

Ellipsis

In [20]:
# model = Sequential()
# model.add(Dense(output_dim = 500, input_dim = max_len, activation= "relu"))
# model.add(Dense(activation = "relu"))
# model.add(Dense(activation = "relu"))
# model.add(Dense(1, activation = "sigmoid"))
# model.summary()

In [21]:
sgd = SGD(learning_rate=0.0001)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_padded, y_train, epochs=30, validation_data=(valid_padded, y_valid),batch_size=32)

Epoch 1/30
72/72 [==============================] - 16s 119ms/step - loss: 0.6928 - accuracy: 0.5366 - val_loss: 0.6929 - val_accuracy: 0.5226
Epoch 2/30
72/72 [==============================] - 7s 97ms/step - loss: 0.6928 - accuracy: 0.5366 - val_loss: 0.6929 - val_accuracy: 0.5226
Epoch 3/30
72/72 [==============================] - 7s 97ms/step - loss: 0.6927 - accuracy: 0.5366 - val_loss: 0.6929 - val_accuracy: 0.5226
Epoch 4/30
72/72 [==============================] - 8s 105ms/step - loss: 0.6927 - accuracy: 0.5366 - val_loss: 0.6929 - val_accuracy: 0.5226
Epoch 5/30
72/72 [==============================] - 7s 99ms/step - loss: 0.6926 - accuracy: 0.5366 - val_loss: 0.6928 - val_accuracy: 0.5226
Epoch 6/30
72/72 [==============================] - 7s 99ms/step - loss: 0.6926 - accuracy: 0.5366 - val_loss: 0.6928 - val_accuracy: 0.5226
Epoch 7/30
72/72 [==============================] - 7s 100ms/step - loss: 0.6926 - accuracy: 0.5366 - val_loss: 0.6928 - val_accuracy: 0.5226
Epoch 8/3

In [22]:
y_prob = model.predict(test_padded)
y_pred = np.argmax(y_prob,axis=1)
print(classification_report(y_test_int,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       598
           1       0.51      1.00      0.68       632

    accuracy                           0.51      1230
   macro avg       0.26      0.50      0.34      1230
weighted avg       0.26      0.51      0.35      1230



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
